In [9]:
import os
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import PIL
from PIL import Image
import numpy as np

In [2]:
#Paths to my folders
train_dir='train'
test_dir='test'
validation_dir='valid'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255, #Rescales pixel values to the range [0, 1] by dividing by 255, the maximum value for RGB
    rotation_range=20, #Randomly rotates images by up to 20 degrees 
    width_shift_range=0.2, #Randomly shifts images horizontally
    height_shift_range=0.2, #Randomly shifts images vertically
    shear_range=0.2, #Applies random shear transformations
    zoom_range=0.2, #Randomly zooms inside images
    horizontal_flip=True, #Randomly flips images horizontally
    fill_mode='nearest' #Fills in new pixels created by the transformations with the nearest pixel values
)

validation_datagen = ImageDataGenerator(rescale=1./255) #Rescales validation images

test_datagen = ImageDataGenerator(rescale=1./255) #Rescales test images

train_generator = train_datagen.flow_from_directory(
    train_dir, #Directory with training images
    target_size=(250, 250), #Resizes images to 250x250 pixels
    batch_size=32, #Generates batches of 32 images
    class_mode='categorical' #Uses categorical labels for classes
)

test_generator = test_datagen.flow_from_directory(
    test_dir, #Directory with test images
    target_size=(250, 250),
    batch_size=32, 
    class_mode='categorical' 
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, #Directory with validation images
    target_size=(250, 250), 
    batch_size=32,
    class_mode='categorical'
)


Found 7946 images belonging to 70 classes.
Found 700 images belonging to 70 classes.
Found 700 images belonging to 70 classes.


In [4]:
#Model creation

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(250,250,3)), #Specify the number of filters and their dimensions
    layers.MaxPooling2D(2,2), #Reduce dimensions while retaining important features
    layers.Conv2D(64, (3,3), activation='relu' ), #Another convolutional layer
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'), #Layer filters
    layers.MaxPooling2D(2,2),
    layers.Flatten(), #Flatten data, converting from 3D to 1D
    layers.Dense(512, activation='relu'), #Fully connected layer
    layers.Dropout(0.5), #Randomly drop neurons to prevent overfitting
    layers.Dense(70, activation='softmax') #Output layer with 15 categories, using softmax activation for multi-class classification
])

f:\data\guess-the-image\mokymai\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
#Preparing model for training
model.compile(
    optimizer=Adam(learning_rate=1e-4), #Specify the optimizer and learning rate
    loss='categorical_crossentropy', #Define the loss function to calculate model's error
    metrics=['accuracy'] 
)

In [7]:
history = model.fit(
    train_generator,
    steps_per_epoch= train_generator.samples // train_generator.batch_size, #Get the integer number of steps per epoch
    epochs=30,
    validation_data= validation_generator,
    validation_steps= validation_generator.samples // validation_generator.batch_size #Avoid division by zero error when batch size is 0
)

Epoch 1/30


f:\data\guess-the-image\mokymai\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


248/248 ━━━━━━━━━━━━━━━━━━━━ 464s 2s/step - accuracy: 0.0206 - loss: 4.2742 - val_accuracy: 0.0432 - val_loss: 4.0669
Epoch 2/30
  1/248 ━━━━━━━━━━━━━━━━━━━━ 5:47 1s/step - accuracy: 0.0312 - loss: 4.0154

C:\Users\Svečias\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0312 - loss: 4.0154 - val_accuracy: 0.0714 - val_loss: 4.0168
Epoch 3/30
248/248 ━━━━━━━━━━━━━━━━━━━━ 439s 2s/step - accuracy: 0.0388 - loss: 4.0859 - val_accuracy: 0.0610 - val_loss: 3.8693
Epoch 4/30
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0312 - loss: 3.8055 - val_accuracy: 0.1429 - val_loss: 3.6621
Epoch 5/30
248/248 ━━━━━━━━━━━━━━━━━━━━ 398s 2s/step - accuracy: 0.0517 - loss: 3.9667 - val_accuracy: 0.1071 - val_loss: 3.6917
Epoch 6/30
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 967us/step - accuracy: 0.0938 - loss: 4.1899 - val_accuracy: 0.0714 - val_loss: 3.7650
Epoch 7/30
248/248 ━━━━━━━━━━━━━━━━━━━━ 444s 2s/step - accuracy: 0.0664 - loss: 3.8799 - val_accuracy: 0.1131 - val_loss: 3.6721
Epoch 8/30
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0938 - loss: 3.9365 - val_accuracy: 0.1071 - val_loss: 3.6455
Epoch 9/30
248/248 ━━━━━━━━━━━━━━━━━━━━ 446s 2s/step - accuracy: 0.0765 - loss: 3.8258 - val_accuracy: 0.1429 

In [8]:
model.save('modelis.h5')

In [10]:
def preprocess_image(image_path):
    img = Image.open(image_path)

    if image_path.endswith('.png'):
        im = Image.open(image_path)
        img = im.convert('RGB')

    img = img.resize((250,250))
    img_array = np.array(img)
    img_array=img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [13]:
class_indices = train_generator.class_indices
class_labels = {value:key for key, value in class_indices.items()} 


def predict_breed(path_to_img):
    img_array = preprocess_image(path_to_img)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return class_labels[predicted_class[0]]

print(predict_breed('chow.jpg'))
print(predict_breed('chow_chow.jpg'))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Rhodesian
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Cockapoo
